# Personal Well-being in the UK

*Personal well-being in the UK: local authority update, 2015 to 2016* https://www.ons.gov.uk/releases/personalwellbeingintheuklocalauthorityupdate2015to2016

Quick notebook to parse out breakdowns of wellbeing metrics by Local Authority.

In [1]:
import pandas as pd

In [5]:
#Download the data
url='https://www.ons.gov.uk/file?uri=/peoplepopulationandcommunity/wellbeing/datasets/headlineestimatesofpersonalwellbeing/localauthorityupdate2015to2016/headlineestimatesofpersonalwellbeinglocalauthorityupdate2015to2016.xls'
fn=url.split('/')[-1]
!wget {url} -O data/{fn}

In [13]:
#Check the sheetnames
xl=pd.ExcelFile('data/{}'.format(fn))
xl.sheet_names

['Contents',
 'Life Satisfaction - Means ',
 'Life Satisfaction - Thresholds',
 'Worthwhile - Means',
 'Worthwhile - Thresholds',
 'Happy - Mean',
 'Happy - Thresholds',
 'Anxiety - Means',
 'Anxiety - Thresholds']

In [363]:
#Function to load in the data from a particular sheet - some wrangling required to handle the multiindex
#eg see http://stackoverflow.com/questions/39745627/setting-a-row-index-on-and-querying-a-pandas-dataframe-with-multi-index-columns
def getData(typ):
    df=pd.read_excel('data/{}'.format(fn),sheetname='{} - Thresholds'.format(typ),header=None,skiprows=4)

    df.dropna(axis=0,inplace=True,how='all')
    df[0][:3]=df[0][:3].fillna(method='backfill',axis=0)
    df[1][:3]=df[1][:3].fillna(method='backfill',axis=0)
    df[:2]=df[:2].fillna(method='ffill',axis=1)
    df=df.T.set_index([0,1]).T
    df=df.rename(columns=lambda x: x.strip())
    df=df.drop(df.index[[0]])

    df.set_index([('Area Codes','Area Codes'),
                  ('Area Names','Area Names')], inplace=True)
    df.index.names = ['Area Codes','Area Names']
    df.sort_index(inplace=True)
    #idx = pd.IndexSlice
    #df.loc[idx['E06000047',:], :]['2011/12*']

    df=df.reset_index()
    df=df[~df['Area Names'].isnull()]
    return df

In [308]:
df.head()

0  Area Codes            Area Names 2011/12*                         2012/13*  \
1                                        Low Medium   High Very High      Low   
7   E06000001            Hartlepool      6.4  19.29  43.95     30.37     8.26   
8   E06000002         Middlesbrough     7.37  16.79   48.1     27.75     7.11   
9   E06000003  Redcar and Cleveland      6.7  19.07   45.6     28.64     6.14   
10  E06000004      Stockton-on-Tees     5.04  16.99  49.98     27.99     5.14   
11  E06000005            Darlington     6.12  20.01  44.19     29.68     6.51   

0                             ...    2013/14*           2014/15*         \
1  Medium   High Very High    ...        High Very High      Low Medium   
7   17.89  47.88     25.96    ...       47.54     28.14     6.67  13.44   
8   19.44  45.45        28    ...       48.72      26.8     5.95  17.97   
9    17.5     45     31.36    ...        47.3     30.38     6.41  12.13   
10  15.73  53.71     25.42    ...       51.59     28.57     6.72  13.29   
11  15.81  49.39      28.3    ...       53.79     26.43     4.28  15.03   

0                   2015/16*                          
1    High Very High      Low Medium   High Very High  
7   47.12     32.78      6.8   16.3  45.43     31.47  
8   46.97     29.12     4.92  13.57  51.93     29.58  
9   50.22     31.24     6.01  15.08  45.75     33.16  
10  46.97     33.02     5.03  15.89  49.03     30.05  
11  50.52     30.17     5.63  14.42   48.5     31.45  

[5 rows x 22 columns]

In [ ]:
#PUT YOUR SEARCH QUERY TERMS HERE
area='Isle of Wight'
period='2014/15*'

In [362]:
from collections import OrderedDict
pc=OrderedDict((b,df[df['Area Names']==area][period][b].iloc[0]) for b in ['Low','Medium','High','Very High'])

clists={'Anxiety':['Very Low','Low','Medium','High'],
        'Life Satisfaction':['Low','Medium','High','Very High'],
        'Happy':['Low','Medium','High','Very High'],
        'Worthwhile':['Low','Medium','High','Very High']
        }

for t in ['Life Satisfaction','Worthwhile','Happy','Anxiety']:
    txtlist=[]
    df=getData(t)
    pc=OrderedDict((b,df[df['Area Names']==area][period][b].iloc[0]) for b in clists[t])
    txtlist.append('''
When it comes to {typ}, in the {period}, the percentage breakdowns for {area} residents were as follows: {pc}
'''.format(typ=t,period=period,area=area,pc=', '.join(['{}: {}%'.format(p,pc[p]) for p in pc])) )
    print('\n'.join(txtlist))


When it comes to Life Satisfaction, in the 2014/15*, the percentage breakdowns for Isle of Wight residents were as follows: Low: 5.41%, Medium: 13.46%, High: 50.73%, Very High: 30.4%


When it comes to Worthwhile, in the 2014/15*, the percentage breakdowns for Isle of Wight residents were as follows: Low: 3.72%, Medium: 11.05%, High: 44.43%, Very High: 40.8%


When it comes to Happy, in the 2014/15*, the percentage breakdowns for Isle of Wight residents were as follows: Low: 8.15%, Medium: 15.64%, High: 37.19%, Very High: 39.03%


When it comes to Anxiety, in the 2014/15*, the percentage breakdowns for Isle of Wight residents were as follows: Very Low: 45.62%, Low: 20.67%, Medium: 15.59%, High: 18.12%

